# Predicting inter-station rapid transit travel times

In [6]:
import time
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import re
import os
import io

import keras
from keras.models import Sequential
from keras.layers import *
from keras.layers.wrappers import *
from keras.callbacks import CSVLogger, EarlyStopping
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as ktf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

%matplotlib inline




print('numpy ver.: ' + np.__version__)
print('pandas ver.: ' + pd.__version__)
print('tensorflow ver.: ' + tf.__version__) 
print('keras ver.: ' + keras.__version__)

We propose to work using state-of-the-art papers to predict the arrival time of a subway car for the following N timesteps. We use the proposed architecture in Petersen et al. (2019) that uses convolutional LSTMs as the basis of the network architecture. In this paper, the authors are trying to predict bus arrival times. To support the use of this kind of network, they compare results with other architecture showing the superiority of their solution. In our work, we will compare LSTM with ConvLSTM architecture. There are some differences between predicting buses and subways, such as traffic congestion. However, we think that the model proposed in the paper could still work as a basis for predicting subway cars’ arrival time.
 
The next steps in our work will be to define some key parameters. We will empirically do this, keeping only the parameters that give the best results. We have to define layers, also if we need encoder/decoder and its sizes, filters size, and if normalization is needed for each layer or only in the initial layer. Also, the dimensions of the kernel could make a difference.

The main drawback of this architecture is that it does not contemplate any other information, such as the difference between weekdays and weekend days, special events that could affect the service, among others. To strengthen our model, we could use the changes proposed in  Agafonov and Yumaganov (2019).

## Load Data

Unfortunately the data as we would like it is not available from MBTA for subway, since they do not store stop events; they only store track circuit events and it is difficult to directly get travel times between stations from that.

Bus stop events are readily available, however, so to start we are testing our methods with a 1 million row sample data from the MBTA number 1 bus in the outbound direction.

This is just for demonstration purposes. We will try to obtain the rail data.

In [10]:
import psycopg2
import ssl
import pandas.io.sql as sqlio

In [11]:
def run_sql(query):
    conn = psycopg2.connect(
        host = 'mbta01.mghpcc.org',
        port = 5432,
        user = 'ehab',
        password = '',
        sslmode = 'verify-full',
        sslrootcert = '../postgresql/root-ca.crt',
        sslcert = '../postgresql/ehab.crt',
        sslkey = '../postgresql/ehab.key',
        database = 'mbta_dw')

    return sqlio.read_sql_query(query, conn)

In [12]:
def get_data(query, csv_file):
    try:
        return pd.read_csv(csv_file)
    except FileNotFoundError:
        df = run_sql(query)
        df.to_csv("csv_file")
        return df

In [13]:
query = """SELECT se.trippieceid, 
    se.seq, 
    se.stopid,
    se.arrival,
    se.departure,
    r.routeid,
    r.routename,
    tp.directionid
FROM avl.stopevent se

join avl.trippiece tp
  on se.trippieceid = tp.trippieceid
join avl.route r
  on r.routeid = tp.routeid

where routename = '01'
  and tp.directionid = 4 -- outbound
  --and arrival > '2018-01-01'

ORDER BY se.trippieceid ASC, seq ASC
limit 1000000
;
"""

In [54]:
stop_events = get_data(query, "stop_events.csv")

In [55]:
stop_events.head()

,trippieceid,seq,stopid,arrival,departure,routeid,routename,directionid
0,25950072,1,64,2015-09-30 09:46:58,2015-09-30 09:46:58,15632,1,4
1,25950077,1,64,2015-09-30 06:45:15,2015-09-30 06:45:46,15632,1,4
2,25950077,5,10003,2015-09-30 06:49:26,2015-09-30 06:49:35,15632,1,4
3,25950077,6,57,2015-09-30 06:50:40,2015-09-30 06:50:40,15632,1,4
4,25950077,8,10590,2015-09-30 06:54:30,2015-09-30 06:54:38,15632,1,4


In [56]:
len(stop_events)

1000000

I am not sure if sequence is unordered or if some stops are just not served. Set the index to a unique identifier based on trippieceid and seq.

In [57]:
stop_events['id'] = stop_events.trippieceid*100 + stop_events.seq

In [58]:
stop_events.set_index('id', inplace=True)

In [59]:
stop_events.head()

,trippieceid,seq,stopid,arrival,departure,routeid,routename,directionid
id,,,,,,,,
2595007201,25950072,1,64,2015-09-30 09:46:58,2015-09-30 09:46:58,15632,1,4
2595007701,25950077,1,64,2015-09-30 06:45:15,2015-09-30 06:45:46,15632,1,4
2595007705,25950077,5,10003,2015-09-30 06:49:26,2015-09-30 06:49:35,15632,1,4
2595007706,25950077,6,57,2015-09-30 06:50:40,2015-09-30 06:50:40,15632,1,4
2595007708,25950077,8,10590,2015-09-30 06:54:30,2015-09-30 06:54:38,15632,1,4


Ensure that sequence is in order:

In [60]:
stop_events.sort_index(inplace=True)

In [61]:
stop_events.head()

,trippieceid,seq,stopid,arrival,departure,routeid,routename,directionid
id,,,,,,,,
2595007201,25950072,1,64,2015-09-30 09:46:58,2015-09-30 09:46:58,15632,1,4
2595007701,25950077,1,64,2015-09-30 06:45:15,2015-09-30 06:45:46,15632,1,4
2595007705,25950077,5,10003,2015-09-30 06:49:26,2015-09-30 06:49:35,15632,1,4
2595007706,25950077,6,57,2015-09-30 06:50:40,2015-09-30 06:50:40,15632,1,4
2595007708,25950077,8,10590,2015-09-30 06:54:30,2015-09-30 06:54:38,15632,1,4


It looks like not all stop events are always reported, perhaps because the bus does not always stop at all stop. This is not a problem that would exist for rail.

In [65]:
# TODO

Convert arrival and departure from str to datetime.

In [62]:
stop_events['arrival'] =  pd.to_datetime(stop_events['arrival'])
stop_events['departure'] =  pd.to_datetime(stop_events['departure'])

## Format data

We must move from the stop events data format as defined by MBTA, to one where each observation of a travel time (in seconds) is identified by the link identified and a timestamp (for example of when the vehicle first entered the link). 

The data format is given below in Petersen et al. (2019).

<img src="input_format.png" alt="Input format for the data" style="width: 450px;"/>

In [67]:
# stop_events['next_arrival'] = stop_events['arrival'].shift(-1)

In [68]:
# stop_events.head(15)

## Define model

In [53]:
timesteps = 32 #decide how many timesteps can we get from the AVL
links = 19 #stations-1
predict = 3 #decide how many steps do we want to predict

model = Sequential()
#normalize input
model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (timesteps, links, 1, 1)))

#encoder
model.add(ConvLSTM2D(name = "convLSTM_1", activation = 'relu', filters = 64, kernel_size = (5,1), padding = 'same', return_sequences = True))
model.add(ConvLSTM2D(name = "convLSTM_2", activation = 'relu', filters = 64, kernel_size = (5,1), padding = 'same', return_sequences = False))

#flatten output and re-organize it for the decoder
model.add(Flatten())
model.add(RepeatVector(predict))
model.add(Reshape((predict, links, 1, 64)))

#decoder
model.add(ConvLSTM2D(name = "convLSTM_3", activation = 'relu', filters = 64, kernel_size = (5,1), padding = 'same', return_sequences = True))
model.add(ConvLSTM2D(name = "convLSTM_4", activation = 'relu', filters = 64, kernel_size = (5,1), padding = 'same', return_sequences = True))
model.add(Dense(units = 1, activation = "relu"))

model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_norm_0 (BatchNormaliza (None, 32, 19, 1, 1)      4         
_________________________________________________________________
convLSTM_1 (ConvLSTM2D)      (None, 32, 19, 1, 64)     83456     
_________________________________________________________________
convLSTM_2 (ConvLSTM2D)      (None, 19, 1, 64)         164096    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1216)              0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 1216)           0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 3, 19, 1, 64)      0         
_________________________________________________________________
convLSTM_3 (ConvLSTM2D)      (None, 3, 19, 1, 64)    

## Train and test

## References

1. Agafonov, A., Yumaganov, A., 2019. "Bus Arrival Time Prediction with LSTM Neural Network", in: Lu, H., Tang, H., Wang, Z. (Eds.), _Advances in Neural Networks – ISNN 2019._ Springer International Publishing, Cham, pp. 11–18. https://doi.org/10.1007/978-3-030-22796-8_2

1. Petersen, N.C., Rodrigues, F., Pereira, F.C., 2019. "Multi-output Bus Travel Time Prediction with Convolutional LSTM Neural Network". _Expert Syst. Appl._ 120, 426–435. https://doi.org/10.1016/j.eswa.2018.11.028

